In [166]:
# IMPORTS

import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow
import lightkurve as lk
import cv2

from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


In [167]:
# PHASE AND BIN FUNCTION

def phase_and_bin(data,period,t0,bin_time):
    data_phased = data.fold(period=period,epoch_time=t0)
    data_binned = data_phased.bin(bin_time)
    return data_binned


In [168]:
# DOWNLOAD (& ADD TO BIG ARRAY)

def download(search):

    lc = search.download()
    lcplot = lc.plot(linewidth=0,marker=".")
    img = cv2.imread(lcplot, 1)
    return img
    

In [169]:
# GET ARRAY OF IMAGE ARRAYS FOR A FILE

def get_lightcurves(filename):

    tbl = pd.read_csv(os.path.abspath(filename),delimiter=",",comment="#")
    
    colnames = tbl.columns.values.tolist()
    if "tid" in colnames:
        TICs = tbl["tid"].astype(str).str[4:]
    elif "tic_id" in colnames:
        TICs = tbl["tic_id"].astype(str).str[4:]
    else:
        print("No TIC ID column found.")

    pics = np.array([])

    for x in range(0,32):
        name = TICs[x]
        search = lk.search_lightcurve("TIC " + name,author="SPOC",sector=1)
        if search.download() is not None:
            pic = download(search)
            pics = np.append(pics,pic)
        
    return(pics)

        

In [170]:
# CREATE TRAIN AND TEST DATASETS

def predictExo(exotrainfile,noexotrainfile,testfile):

    exotraindata = np.array(get_lightcurves(exotrainfile),dtype="float") / 255.0
    noexotraindata = np.array(get_lightcurves(noexotrainfile),dtype="float") / 255.0
    testdata = np.array(get_lightcurves(testfile))

    print(np.shape(exotraindata))

    exolabels = np.ones(32)
    noexolabels = np.zeros(32)

    traindata = np.concatenate((exotraindata,noexotraindata))
    trainlabels = np.concatenate((exolabels,noexolabels))

    # BUILD CNN MODEL

    model = keras.Sequential()

    model.add(keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(64,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(512,activation="relu"))

    model.add(keras.layers.Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    # TRAIN DATA

    model.fit(
        traindata,
        trainlabels,
        batch_size = 1,
        steps_per_epoch = 250,
        epochs = 10,
    )

    for y in testfile:

        Y = get_lightcurves(testfile[y])
        pic = Image.fromarray(Y)
        pic.show()
        X = np.expand_dims(Y,axis=0)

        val = model.predict(X)
        print(val)

        if val == 1:
            plt.xlabel("Exoplanet detected!",fontsize=30)
        elif val == 0:
            plt.xlabel("No exoplanet detected.",fontsize=30)

In [171]:
predictExo("PS_2022.06.27_08.12.38.csv","TOI_2022.06.29_08.07.35.csv","PS_2022.07.04_02.32.20.csv")

(32,)
(32,)
Epoch 1/10


/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(
/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(
/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(
/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/lightkurve/search.py:341: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


ValueError: in user code:

    File "/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/vanillacosmos/miniconda3/envs/codeastrogit/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    Fil